<h2>This notebook will be mainly used for the capstone project</h2>

In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup 

In [2]:
res = requests.get("https://postcodes-australia.com/state-postcodes/vic")
soup = BeautifulSoup(res.content,'lxml')
#soup = BeautifulSoup(res.text, 'html.parser')
uls = soup.find_all("ul",{"class":"pclist"})


In [3]:
#https://www.property.com.au/buy
lis = []
strLink = ''
strLi = ''
for ul in uls:
    for li in ul.findAll('li'):
        if li.find('ul'):
            strLink = li.find('a').string
            strLi = li.find('li').string
            lis.append((strLink, strLi))

data = pd.DataFrame(lis, columns = ('PostalCode', 'Neighborhood'))
data.tail(20)


,PostalCode,Neighborhood
741,8383,Melbourne
742,8386,Melbourne
743,8388,Melbourne
744,8390,Melbourne
745,8393,Melbourne
746,8394,Melbourne
747,8396,Melbourne
748,8399,Melbourne
749,8500,North Melbourne
750,8507,Carlton


In [4]:
data['Neighborhood'] = data[['PostalCode','Neighborhood']].groupby(['PostalCode'])['Neighborhood'].transform(lambda x: ','.join(x))
result = data[['PostalCode','Neighborhood']].drop_duplicates().reset_index(drop=True)
result.head(20)

,PostalCode,Neighborhood
0,3000,Melbourne
1,3001,Melbourne
2,3002,East Melbourne
3,3003,West Melbourne
4,3004,Melbourne
5,3005,World Trade Centre
6,3006,Southbank
7,3008,Docklands
8,3010,University Of Melbourne
9,3011,Footscray


In [5]:
result.shape

(761, 2)

In [34]:
#CSV file from http://www.corra.com.au/australian-postcode-location-data/
df = pd.read_csv('Australian_Post_Codes_Lat_Lon.csv')
df.rename(columns={'postcode':'PostalCode'}, inplace=True)
df.head()

,PostalCode,suburb,state,dc,type,lat,lon
0,200,AUSTRALIAN NATIONAL UNIVERSITY,ACT,AUSTRALIAN NATIONAL UNI LPO,Post Office Boxes,-35.277272,149.117136
1,221,BARTON,ACT,NaN,LVR,-35.201372,149.095065
2,800,DARWIN,NT,DARWIN DELIVERY CENTRE,Delivery Area,-12.801028,130.955789
3,801,DARWIN,NT,DARWIN DELIVERY CENTRE,Post Office Boxes,-12.801028,130.955789
4,804,PARAP,NT,PARAP,Post Office Boxes,-12.432181,130.843310


In [55]:
latlong=[]
currentPC = ''
for index, row in df.iterrows():
    if(row['state'] == 'VIC'):
        if(currentPC != row['PostalCode']):
            latlong.append((row['PostalCode'], row['suburb'], row['type'], row['lat'], row['lon']))
        currentPC = row['PostalCode']

latlong_data = pd.DataFrame(latlong, columns = ('PostalCode', 'Suburb', 'Type', 'Latitude', 'Longitude'))
latlong_data.shape

(746, 5)

In [35]:
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install folium
import folium # map rendering library

You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [72]:
address = 'Victoria, Australia'

geolocator = Nominatim(user_agent="melbourne_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Victoria are {}, {}.'.format(latitude, longitude))
latlong_data.head(20)

The geograpical coordinate of Victoria are -36.5986096, 144.6780052.


,PostalCode,Suburb,Type,Latitude,Longitude
0,3000,MELBOURNE,Delivery Area,-37.814563,144.970267
1,3001,MELBOURNE,Post Office Boxes,-38.365017,144.765920
2,3002,EAST MELBOURNE,Delivery Area,-37.816640,144.987811
3,3003,WEST MELBOURNE,Delivery Area,-37.806255,144.941123
4,3004,MELBOURNE,Delivery Area,-37.837324,144.976335
5,3005,WORLD TRADE CENTRE,Delivery Area,-37.822262,144.954856
6,3006,SOUTHBANK,Delivery Area,-37.823258,144.965926
7,3008,DOCKLANDS,Delivery Area,-37.814719,144.948039
8,3010,UNIVERSITY OF MELBOURNE,Delivery Area,-37.796152,144.961351
9,3011,FOOTSCRAY,Delivery Area,-37.799770,144.899587


In [70]:
# create map of Melbourne using latitude and longitude values
map_melbourne = folium.Map(location=[latitude, longitude], zoom_start=7)

# add markers to map
for lat, lng, suburb in zip(latlong_data['Latitude'], latlong_data['Longitude'], latlong_data['Suburb']):
    label = '{}, {}'.format(latlong_data, suburb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne)  
    
map_melbourne

In [79]:
#https://www.domain.com.au/suburb-profile/west-melbourne-vic-3003
res = requests.get("https://www.domain.com.au/suburb-profile/west-melbourne-vic-3003")
soup = BeautifulSoup(res.text, 'html.parser')
table = soup.find_all('table')[0] 


In [86]:
item = []
for tbody in table:
    for td in tbody.findAll("td", class_="css-1r1wsml"):
        if(strin(td.text) == '2'):
            twoHouse = td.text


 2
$820k
 3
-
 4
-
 1
$392k
 2
$603k
 3
-


In [87]:
#https://www.domain.com.au/suburb-profile/west-melbourne-vic-3003
res = requests.get("https://www.domain.com.au/suburb-profile/east-melbourne-vic-3002")
soup = BeautifulSoup(res.text, 'html.parser')
table = soup.find_all('table')[0] 

In [88]:
item = []
for tbody in table:
    for td in tbody.findAll("td", class_="css-1r1wsml"):
        print(td.text)

 2
-
 3
-
 4
-
 5
-
 1
$490k
 2
$898k
 3
$1.81m


In [ ]:
#https://www.kaggle.com/anthonypino/melbourne-housing-market#Melbourne_housing_FULL.csv